# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제

In [3]:
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


In [4]:
df1 = pd.DataFrame({"age" : ["baby", "baby", "baby", "baby"],
                   "income" : ["high", "high", "medium", "low"],
                   "student" : ["no", "yes", "no", "yes"],
                   "credit_rating" : ["fair", "excellent", "fair", "fair"],
                   "class_buys_computer" : ["no", "no", "no", "no"]})

In [5]:
pd_data = pd_data.append(df1)

# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [6]:
def get_gini(df, label):
    value_list = list(df[label].value_counts()) # 타겟변수의 각 클래스를 가진 행의 갯수
    gini_d = np.array([(value_list[value] / sum(value_list)) for value in range(len(value_list))]) # 각 클래스의 지니계수
    gini = 1 - sum(gini_d**2) # 총 지니계수
    return gini

In [7]:
get_gini(pd_data,'class_buys_computer')

0.5

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [8]:
import itertools # 변수의 모든 클래시 조합을 얻기 위해 itertools 불러오기
def get_binary_split(df, attribute):
    classes = list(pd.unique(df[attribute])) # 클래스를 리스트로 받아옴
    result = [] 
    for L in range(0, len(classes)+1): # 0개부터 L개 까지의 조합을 얻기 위한 for문
        for subset in itertools.combinations(classes, L): # L개의 원소로 이루어진 클래스의 부분집합
            result.append(list(subset))
    if len(classes) == 1:
        return(classes)
    return result[1:-1] 

In [9]:
get_binary_split(pd_data, "age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['baby'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['youth', 'baby'],
 ['middle_aged', 'senior'],
 ['middle_aged', 'baby'],
 ['senior', 'baby'],
 ['youth', 'middle_aged', 'senior'],
 ['youth', 'middle_aged', 'baby'],
 ['youth', 'senior', 'baby'],
 ['middle_aged', 'senior', 'baby']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [10]:
def get_attribute_gini_index(df, attribute, label):
    binary = get_binary_split(df, attribute)
    if len(binary) > 1 : 
        classes = list(pd.unique(df[attribute]))
        class_list=list()
        g=list()
        for i in binary : 
            gini_d1 = df.loc[df[attribute].isin(i)] # 해당 클래스 조합에 속한 클래스를 가진 데이터들
            gini_d2 = df.loc[~df[attribute].isin(i)] # 위의 것을 제외한 데이터들
            gini = (get_gini(gini_d1, label) * len(gini_d1) / len(df)) + (get_gini(gini_d2, label) * len(gini_d2) / len(df)) # 지니계수 계산
            class_list.append(tuple(i))
            g.append(gini)
        result = dict(zip(class_list,g)) # 딕셔너리 형태로 내보내기
    else : 
        result = dict(zip(pd.unique(df[attribute]), [0.5])) # class가 한개로 이루어진 변수의 경우 지니계수가 의미가 없으므로 0.5 반환
    return result

In [11]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{('youth',): 0.49230769230769234,
 ('middle_aged',): 0.3571428571428571,
 ('senior',): 0.49230769230769234,
 ('baby',): 0.3571428571428571,
 ('youth', 'middle_aged'): 0.4444444444444444,
 ('youth', 'senior'): 0.5,
 ('youth', 'baby'): 0.345679012345679,
 ('middle_aged', 'senior'): 0.345679012345679,
 ('middle_aged', 'baby'): 0.5,
 ('senior', 'baby'): 0.4444444444444444,
 ('youth', 'middle_aged', 'senior'): 0.3571428571428571,
 ('youth', 'middle_aged', 'baby'): 0.49230769230769234,
 ('youth', 'senior', 'baby'): 0.3571428571428571,
 ('middle_aged', 'senior', 'baby'): 0.49230769230769234}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [12]:
my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - ('youth', 'baby') : 0.345679012345679


# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [10]:
##문제1 답안
get_binary_split(pd_data, "income")

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

In [11]:
##문제2 답안
columns = list(pd_data)[:-1] # 타겟변수를 제외한 변수의 이름 받기
for i in columns: 
    print(i + ":")
    print(min(get_attribute_gini_index(pd_data, i, "class_buys_computer").items()))

## age가 가장 낮은 Gini index를 가지고 있으므로 age가 가장 중요한 변수이며, Gini index는 0.4429이다.

age:
(('middle_aged',), 0.35714285714285715)
income:
(('high',), 0.44285714285714284)
student:
(('no',), 0.3673469387755103)
credit_rating:
(('excellent',), 0.42857142857142855)


In [13]:
##문제3 답안
pd_data_middle = pd_data.loc[pd_data["age"] == "middle_aged"]
for i in columns:
    print(i + ":")
    print(min(get_attribute_gini_index(pd_data_middle, i, "class_buys_computer").items()))
#age가 middle_aged인 경우 타겟 변수인 class_buys_computer의 값이 모두 yes로 일정하기 때문에 다른 변수들이 의미없다.

age:
('middle_aged', 1.0)
income:
(('high',), 0.0)
student:
(('no',), 0.0)
credit_rating:
(('excellent',), 0.0)


In [15]:
pd_data_notmiddle = pd_data.loc[pd_data["age"] != "middle_aged"]
columns = list(pd_data)[:-1]
for i in columns:
    print(i + ":")
    print(min(get_attribute_gini_index(pd_data_notmiddle, i, "class_buys_computer").items()))
#age가 senior 혹은 youth일 경우 student가 중요한 변수이며, 지니계수는 0.32 이다.

age:
(('senior',), 0.48)
income:
(('high',), 0.375)
student:
(('no',), 0.31999999999999984)
credit_rating:
(('excellent',), 0.41666666666666663)
